# Image Recognition
### Mar 2021
* Instructor: Arnab Bose
* Author: Elly Yang

In [ ]:
from pyimagesearch import config
from imutils import paths
import random
import shutil
import os

# input & reshuffle images
imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

# train, test, validation split
i = int(len(imagePaths) * config.TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

i = int(len(trainPaths) * config.VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

datasets = [("training", trainPaths, config.TRAIN_PATH), 
            ("validation", valPaths, config.VAL_PATH), 
            ("testing", testPaths, config.TEST_PATH)]

for (dType, imagePaths, baseOutput) in datasets:
    print("[INFO] building '{}' split".format(dType))
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)
    for inputPath in imagePaths:
        filename = inputPath.split(os.path.sep)[-1]
        label = inputPath.split(os.path.sep)[-2]
        labelPath = os.path.sep.join([baseOutput, label])
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)
        shutil.copy2(inputPath, os.path.sep.join([labelPath, filename]))

# initialize train, test, validation generator
from keras.preprocessing.image import ImageDataGenerator
BS = 200 

trainAug = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest")

datagen = ImageDataGenerator(rescale=1./255)
trainGen = trainAug.flow_from_directory(config.TRAIN_PATH, class_mode="binary", target_size=(150, 150), batch_size=BS)
valGen = datagen.flow_from_directory(config.VAL_PATH, class_mode="binary", target_size=(150, 150), batch_size=BS)
testGen = datagen.flow_from_directory(config.TEST_PATH, class_mode="binary", target_size=(150, 150), batch_size=BS)

# extract features & labels
import numpy as np
from keras.applications import VGG16
conv_base = VGG16(weights="imagenet", include_top=False)

def get_features(xGen):
    sample_count = xGen.samples
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    i = 0
    for inputs_batch, labels_batch in xGen:
        features[i * BS: (i+1) * BS] = conv_base.predict(inputs_batch)
        labels[i * BS: (i+1) * BS] = labels_batch
        i += 1
        if ((i * BS % 1000) == 0): print("processed size =", i * BS)
        if i * BS >= sample_count: break
    return features, labels

train_features, train_labels = get_features(trainGen)
np.save('train_features', train_features)
np.save('train_labels', train_labels)

val_features, val_labels = get_features(valGen)
np.save('val_features', val_features)
np.save('val_labels', val_labels)

test_features, test_labels = get_features(testGen)
np.save('test_features', test_features)
np.save('test_labels', test_labels)

## load features & labels

In [1]:
import numpy as np
train_features = np.load('train_features.npy')
train_labels = np.load('train_labels.npy')
val_features = np.load('val_features.npy')
val_labels = np.load('val_labels.npy')
test_features = np.load('test_features.npy')
test_labels = np.load('test_labels.npy')

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Input, Flatten
model = Sequential()
model.add(Flatten())
model.add(Dense(256, activation='relu', input_dim=512))
model.add(Dropout(rate=.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
history = model.fit(train_features, train_labels, epochs=100, batch_size=200, validation_data=(val_features, val_labels))
loss, accuracy = model.evaluate(test_features, test_labels)
print('accuracy:', round(accuracy*100, 1))

Epoch 1/100
100/100 [==============================] - 3s 23ms/step - loss: 0.9404 - acc: 0.6588 - val_loss: 0.3420 - val_acc: 0.8775
Epoch 2/100
100/100 [==============================] - 2s 17ms/step - loss: 0.3573 - acc: 0.8401 - val_loss: 0.3076 - val_acc: 0.8648
Epoch 3/100
100/100 [==============================] - 2s 17ms/step - loss: 0.3007 - acc: 0.8736 - val_loss: 0.3597 - val_acc: 0.8534
Epoch 4/100
100/100 [==============================] - 2s 17ms/step - loss: 0.2633 - acc: 0.8959 - val_loss: 0.2323 - val_acc: 0.9074
Epoch 5/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2472 - acc: 0.9012 - val_loss: 0.2738 - val_acc: 0.8938
Epoch 6/100
100/100 [==============================] - 2s 20ms/step - loss: 0.2361 - acc: 0.9074 - val_loss: 0.2344 - val_acc: 0.9161
Epoch 7/100
100/100 [==============================] - 3s 31ms/step - loss: 0.2137 - acc: 0.9191 - val_loss: 0.2120 - val_acc: 0.9138
Epoch 8/100
100/100 [==============================] - 2s 19ms